In [14]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
from torch.nn import functional as F
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from tqdm import tqdm
import numpy as np
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel, AdamW

In [15]:
# Load dataset
data = pd.read_excel('/kaggle/input/ubmec-bangla-dataset/UBMEC.xlsx')



In [16]:
# Preprocess the dataset
data = data.dropna(subset=['text'])  # Drop rows where 'text' column is NaN
texts = data['text'].astype(str).values  # Ensure all texts are strings
labels = data['classes'].values

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# # Preprocess the dataset
# texts = data['text'].values
# labels = data['classes'].values

# # Encode the labels
# label_encoder = LabelEncoder()
# labels = label_encoder.fit_transform(labels)

# Train-test split
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Split training data further for validation
texts_train, texts_val, labels_train, labels_val = train_test_split(texts_train, labels_train, test_size=0.1, random_state=42)

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class BanglaDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }





In [17]:
# Create DataLoaders
max_length = 512
batch_size = 8

train_dataset = BanglaDataset(texts_train, labels_train, tokenizer, max_length)
val_dataset = BanglaDataset(texts_val, labels_val, tokenizer, max_length)
test_dataset = BanglaDataset(texts_test, labels_test, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class BertTextClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BertTextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        output = self.dropout(pooled_output)
        return self.fc(output)

# Example model parameters
num_classes = len(label_encoder.classes_)
model = BertTextClassifier(num_classes)

# Training parameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=3, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.best_loss = np.inf
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f'Early stopping triggered after {self.counter} epochs with no improvement.')

# Training loop with validation and early stopping
epochs = 20
early_stopping = EarlyStopping(patience=3, verbose=True)

for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            val_loss += criterion(outputs, labels).item()
    
    val_loss /= len(val_loader)  # Average validation loss

    print(f'Epoch {epoch + 1}/{epochs} - Validation Loss: {val_loss}')

    # Check early stopping
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 135/135 [00:18<00:00,  7.33it/s]


Epoch 1/20 - Validation Loss: 1.34835280224129


100%|██████████| 135/135 [00:18<00:00,  7.32it/s]


Epoch 2/20 - Validation Loss: 1.2453252037366231


...

Epoch 3/20 - Validation Loss: 1.262376062075297


100%|██████████| 135/135 [00:18<00:00,  7.34it/s]


Epoch 4/20 - Validation Loss: 1.2954042295614878


100%|██████████| 135/135 [00:18<00:00,  7.33it/s]

Epoch 5/20 - Validation Loss: 1.5137458461302298
Early stopping triggered after 3 epochs with no improvement.
Early stopping


In [21]:
# Evaluation before quantization
model.to('cpu')
device = 'cpu'
model.eval()
test_preds = []
test_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask)  # Pass both input_ids and attention_mask
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())

test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class, average='weighted')
precision = precision_score(test_labels, test_preds_class, average='weighted')
f1 = f1_score(test_labels, test_preds_class, average='weighted')
micro_f1 = f1_score(test_labels, test_preds_class, average='micro')
macro_roc_auc = roc_auc_score(test_labels, test_preds, multi_class='ovo', average='macro')

print(f'Before Quantization - Accuracy: {accuracy}')
print(f'Before Quantization - Recall: {recall}')
print(f'Before Quantization - Precision: {precision}')
print(f'Before Quantization - F1: {f1}')
print(f'Before Quantization - Micro F1: {micro_f1}')
print(f'Before Quantization - Macro Roc Auc: {macro_roc_auc}')

# # Evaluation on test set
# model.eval()
# test_loss = 0
# correct_predictions = 0
# with torch.no_grad():
#     for batch in tqdm(test_loader):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids, attention_mask)
#         test_loss += criterion(outputs, labels).item()
#         _, preds = torch.max(outputs, dim=1)
#         correct_predictions += torch.sum(preds == labels)

# test_loss /= len(test_loader)  # Average test loss
# accuracy = correct_predictions.double() / len(test_loader.dataset)
# print(f'Test Loss: {test_loss}, Test Accuracy: {accuracy}')

100%|██████████| 336/336 [29:33<00:00,  5.28s/it]

Before Quantization - Accuracy: 0.5252976190476191
Before Quantization - Recall: 0.5252976190476191
Before Quantization - Precision: 0.5534137028746428
Before Quantization - F1: 0.5332314913791076
Before Quantization - Micro F1: 0.5252976190476191
Before Quantization - Macro Roc Auc: 0.827190443483228


In [23]:
# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {nn.Linear},
    dtype=torch.qint8
)

# Evaluation after quantization
test_preds = []
test_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = quantized_model(input_ids, attention_mask)  # Pass both input_ids and attention_mask
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())

test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class, average='weighted')
precision = precision_score(test_labels, test_preds_class, average='weighted')
f1 = f1_score(test_labels, test_preds_class, average='weighted')
micro_f1 = f1_score(test_labels, test_preds_class, average='micro')
macro_roc_auc = roc_auc_score(test_labels, test_preds, multi_class='ovo', average='macro')

print(f'After Quantization - Accuracy: {accuracy}')
print(f'After Quantization - Recall: {recall}')
print(f'After Quantization - Precision: {precision}')
print(f'After Quantization - F1: {f1}')
print(f'After Quantization - Micro F1: {micro_f1}')
print(f'After Quantization - Macro Roc Auc: {macro_roc_auc}')

100%|██████████| 336/336 [27:41<00:00,  4.95s/it]

After Quantization - Accuracy: 0.5219494047619048
After Quantization - Recall: 0.5219494047619048
After Quantization - Precision: 0.5477307224560475
After Quantization - F1: 0.5273914640448607
After Quantization - Micro F1: 0.5219494047619048
After Quantization - Macro Roc Auc: 0.8261630369547507
